## In order to run code successfully please upload the data set_a, set B, code 2.1, 2.2 and 2.3 in one folder of jupyter notebook.
## For question 2.1 the output is the in form of word. Prentropy is the entropy befroe split. And the entropy is that after split. The index 0,1,2,3 denotes feature sepal length (cm)', 'sepal width (cm)', 'petal length (cm)' and 'petal width (cm). 

In [1]:
import pandas as pd
import numpy as np
iris = pd.read_csv('set_a.csv',header=None)

# 数据特征：150行， 4列
features=iris.iloc[:,0:4].values
target=iris.iloc[:,4].values
target=target.astype(int)

iris.feature_names=['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
feature_names=iris.feature_names
iris.target_names=['0','1','2']
class_names = iris.target_names

iris_count = np.zeros(3)

iris_count[0] = target[target == 0].size
iris_count[1] = target[target == 1].size
iris_count[2] = target[target == 2].size

print("三种鸢尾花的数量分别为：", iris_count)

# 计算每种鸢尾花的概率
iris_probability = np.divide(iris_count, 100)

print("三种莺尾花的概率为：", iris_probability)

# 根据公式算出鸢尾花的熵：
iris_h = -np.sum(iris_probability * np.log2(iris_probability))

print("鸢尾花的熵为：", iris_h)

def calcEntropy(target):
    label = np.unique(target)
    n = label.size
    count = np.zeros(n)
    p_i = np.zeros(n)
    for i in range(n):
        count[i] = target[target == label[i]].size
    
    # 计算每个类别的概率
    p_i = np.divide(count, target.size)
    
    # 计算熵
    entropy = 0
    for i in range(n):
        entropy = entropy - p_i[i] * np.log2(p_i[i])
    
    return entropy

def calcConditionEntropy(feature, condition, target):
    true_condition = condition(feature)
    false_condition = true_condition == False
    target_true = target[true_condition]
    target_false = target[false_condition]
    # 每种属性类别的数量除以总数就计算出其概率
    p_true = target_true.size / target.size
    p_false = 1 - p_true
    # 每种属性类别的概率乘以该类别下的信息熵
    entropy = p_true * calcEntropy(target_true) + p_false * calcEntropy(target_false)
    return entropy


# 重新使用函数计算一次鸢尾花的熵，结果跟上面一样
H = calcEntropy(target)

# 加入鸢尾花花瓣宽度属性后，计算鸢尾花的条件熵
petal_length = features[:,2]
HC = calcConditionEntropy(petal_length, lambda feature: feature <= 2.45, target)
print('鸢尾花默认的信息熵 ：', H)
print('带花瓣宽度的条件熵 ：', HC)

def generate_feature_points(feature, target):
    """
    生成特征的所有分界点: 先对特征进行排序，然后将 target 有变动的地方作为分界点
    :param feature: 一维数组，一个特征的样本数据
    :param target: 一维数组，数字或者字符串的分类标签
    :return: 包含所有分界点的一维数组
    """

    argsort = feature.argsort()
    f1 = feature[argsort]
    t1 = target[argsort]

    last_value = target[0]
    split_value = []

    # 找出所有分裂点
    for i in range(t1.size):
        if last_value != t1[i]:
            split_value.append((f1[i] + f1[i - 1]) / 2)
            last_value = t1[i]

    return np.array(split_value)


def calc_feature_entropy(feature, target):
    """
    计算一个特征的所有分界点的条件熵，返回最小的那个条件熵（条件熵越小，信息增益越大）
    :param feature: 一维数组，一个特征的样本数据
    :param target: 一维数组，数字或者字符串的分类标签
    :return: 分界点和条件熵
    """
    min_entropy = float('inf')
    min_point = 0
    prentropy=calcEntropy(target)
    points = generate_feature_points(feature, target)
    for p in points:
        entropy = calcConditionEntropy(feature, lambda f: f <= p, target)
        if entropy < min_entropy:
            min_entropy = entropy
            min_point = p

    '没有分界点说明只有一类数据标签，熵为0'
    if points.size == 0:
        min_entropy = 0

    return min_point, min_entropy,prentropy

calc_feature_entropy(features[:,2], target)

def select_feature(features, target):
    """
    从所有特征中选择出条件熵最小的特征（即最大增益）
    :param features: 二维数据，包含所有特征的样本数据
    :param target: 一维数组，数字或者字符串的分类标签
    :return: 特征索引，条件熵，特征分界点
    """
    min_entropy = float('inf')
    min_point = 0
    num = features.shape[1]
    index = 0
    for i in range(num):
        point, entropy,prentropy1 = calc_feature_entropy(features[:, i], target)
        if entropy <= min_entropy:
            index = i
            min_point = point
            min_entropy = entropy
            prentropy=prentropy1

    return index, min_point, min_entropy,prentropy

class TreeNode:
    
    feature_index = ''
    feature_point = 0
    feature_entropy = 0
    target_label = ''
    true_node = None
    false_node = None
    feature_right = ''
    target_right = ''
    feature_left = ''
    target_left = ''
    right_entropy=0
    leaft_entropy=0
    

    @staticmethod
    def decision(feature, point):
        return feature < point

三种鸢尾花的数量分别为： [21. 25. 54.]
三种莺尾花的概率为： [0.21 0.25 0.54]
鸢尾花的熵为： 1.4528662323792307
鸢尾花默认的信息熵 ： 1.4528662323792307
带花瓣宽度的条件熵 ： 0.9399241705336179


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  # Remove the CWD from sys.path while we load stuff.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  if sys.path[0] == '':


In [59]:
def build_tree(features, target):
    """
    递归构建决策树
    :param features: 二维数据，包含所有特征的样本数据
    :param target: 一维数组，数字或者字符串的分类标签
    :param idn: 决策树节点 id，通过 id 观察决策树计算过程
    :return: 决策树根节点
    """
    node = TreeNode()

    '选择条件熵最小的特征'
    index, point, entropy,prentropy = select_feature(features, target)
    
    node.feature_index = index
    node.feature_point = point
    node.feature_entropy = entropy
    node.feature_prentropy=prentropy
    
    print('build tree node index %d, point %f, entropy %f,prentropy %f ' %
          ( index, point, entropy,prentropy))
   
    
    
    f_copy = features.copy()
    t_copy = target.copy()
    f = f_copy[:, index]
    selector = node.decision(f, point)
    
    
    left_entropy=calcEntropy(t_copy[selector])
    right_entropy=calcEntropy(t_copy[selector == False])
    
    i=0
    
    if left_entropy == 0 and right_entropy != 0:
        #print('build tree node, index %d, entropy %f ' %
         # ( index ,left_entropy),'left=0')
        #print('right','i value',i)
        build_tree(f_copy[selector == False], t_copy[selector == False])
        #print('split right')
        
    elif right_entropy == 0 and left_entropy != 0:
        #print('build tree node, index %d,  entropy %f ' %
          #( index, right_entropy),'right=0')
        #print('left','i value',i)
        build_tree(f_copy[selector, :], t_copy[selector])
        
    elif right_entropy != 0 and left_entropy !=0:
       
        #print('generate left node from not 0')
        #print(node.point,'1')
        build_tree(f_copy[selector, :], t_copy[selector])
        
        
        #print('generate right node from not0')
        #print(node.point,'2')
        build_tree(f_copy[selector == False], t_copy[selector == False])
        #print(node.point,'3')
        
    
    elif right_entropy == 0 and left_entropy ==0:
        print('build tree node, index %d,  right_entropy %f ' %
          ( index, right_entropy))
        print('build tree node, index %d,  left_entropy %f ' %
          ( index, right_entropy))
        return node     
#'构建决策树，节点编号从 1 开始，深度优先递归方式创建树'
clf=build_tree(features, target)

build tree node index 2, point 2.450000, entropy 0.939924,prentropy 1.452866 
build tree node index 1, point 3.050000, entropy 0.631921,prentropy 0.721928 
build tree node index 2, point 1.150000, entropy 0.723983,prentropy 0.831474 
build tree node index 1, point 3.200000, entropy 0.634945,prentropy 0.764205 
build tree node index 0, point 5.150000, entropy 0.595973,prentropy 0.672295 
build tree node index 0, point 5.250000, entropy 0.600021,prentropy 0.779350 
build tree node index 2, point 1.550000, entropy 0.404563,prentropy 0.650022 
build tree node index 3, point 1.750000, entropy 0.550978,prentropy 0.970951 
build tree node index 3, point 1.250000, entropy 0.666667,prentropy 0.918296 
build tree node index 3, point 0.650000, entropy 0.000000,prentropy 1.000000 
build tree node, index 3,  right_entropy 0.000000 
build tree node, index 3,  left_entropy 0.000000 
build tree node index 2, point 4.550000, entropy 0.775622,prentropy 1.012590 
build tree node index 3, point 1.650000, 